# Uptake Reactions
This notebook adds uptake reactions for [phenol](https://www.genome.jp/dbget-bin/www_bget?C00146), [vanillate](https://www.genome.jp/dbget-bin/www_bget?C06672), [4-hydroxybenzoate](https://www.genome.jp/dbget-bin/www_bget?C00156), [guaiacol](https://www.genome.jp/dbget-bin/www_bget?C01502), and [benzoate](https://www.genome.jp/dbget-bin/www_bget?C00180) to the *R.opacus* genome scale model.

### The model was made using CarveMe and the commands:
The .faa assembly file was downloaded from [NCBI](https://www.ncbi.nlm.nih.gov/assembly/GCF_020542785.1). 
<ol>
    <li>carve r_opacus_bologna.faa -u grampos -o r_opacus_bologna_raw.xml</li>
    <li>gapfill r_opacus_bologna.xml -m M9,LB -o r_opacus_bologna_gapfilled.xml</li>
</ol>


###  Method
<ol>
    <li>Add metabolites neccessary for aromatic growth</li>
    <li>Add reactions to connect the metabolites to the metabolic network</li>
</ol>

###  Required Additional Metabolites
<ul>
    <li>Phenol: extracellular phenol and cytosolic phenol</li>
    <li>Vanillate: extracellular vanillate</li>
    <li>4-Hydroxybenzoate: none</li>
    <li>Guaiacol: extracellular guaiacol and cytosolic guaiacol</li>
    <li>Benzoate: none</li>
</ul>

###  Required Additional Reactions
<ul>
    <li>Phenol: phenol exchange, phenol transport, and phenol monooxygenase</li>
    <li>Vanillate: vanillate exchange, vanillate transport</li>
    <li>4-Hydroxybenzoate: none</li>
    <li>Guaiacol: guaiacol exchange and guaiacol transport, and guaicaol O-demethylase</li>
    <li>Benzoate: none</li>
</ul>

In [1]:
import cobra

### Load genome-scale model
Get *R.opacus* NCBI model generated by CarveMe

In [3]:
model = cobra.io.read_sbml_model("../models/r_opacus_bologna_gapfilled.xml")
model

Name,italian_3
Memory address,0x07f0f21419c90
Number of metabolites,1947
Number of reactions,3011
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"


### Add neccessary metabolites to the model

In [4]:
# Extracellular phenol 
phenol_e = cobra.Metabolite(
    'phenol_e',
    formula='C6H6O',
    name='Phenol',
    compartment='C_e')

# Cytosolic phenol
phenol_c = cobra.Metabolite(
    'phenol_c',
    formula='C6H6O',
    name='Phenol',
    compartment='C_c')

# Extracellular 4-hydroxybenzoate
hbz_e = cobra.Metabolite(
    '4hbz_e',
    formula='C7H5O3',
    name='4-Hydroxybenzoate',
    compartment='C_e')

# Extracellular guaiacol
guaiacol_e = cobra.Metabolite(
    'guaiacol_e',
    formula='C7H8O2',
    name='Guaiacol',
    compartment='C_e')

# Cytosolic guaiacol
guaiacol_c = cobra.Metabolite(
    'guaiacol_c',
    formula='C7H8O2',
    name='Guaiacol',
    compartment='C_c')

# 4-Carboxymuconolactone
cml_4 = cobra.Metabolite(
    '4cml_c',
    formula='C7H4O6',
    name='4-Carboxymuconolactone',
    compartment='C_c')

Add reactions for phenol exchange, transport, and [conversion to catechol](https://www.kegg.jp/entry/R10043) to the model

In [5]:
phenol_exchange = cobra.Reaction('EX_phenol_e')
phenol_exchange.name = 'Phenol exchange'
phenol_exchange.lower_bound = 0.  
phenol_exchange.upper_bound = 1000.  
phenol_exchange.add_metabolites({
    phenol_e: -1.0,
})

phenol_transport = cobra.Reaction('phenol_transport')
phenol_transport.name = 'Phenol transport'
phenol_transport.lower_bound = 0. 
phenol_transport.upper_bound = 1000. 
phenol_transport.add_metabolites({
    phenol_e: -1.0,
    phenol_c:  1.0,
}) 

# EC 1.14.13.244
phenol_monooxygenase = cobra.Reaction('PHEMOX')
phenol_monooxygenase.name = 'Phenol monooxygenase'
phenol_monooxygenase.lower_bound = 0.
phenol_monooxygenase.upper_bound = 1000.  # This is the default
phenol_monooxygenase.add_metabolites({
    phenol_c: -1.0,
    model.metabolites.get_by_id('nadh_c'):   -1.0,
    model.metabolites.get_by_id('h_c'): -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('nad_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
}) 


model.add_reactions([phenol_exchange])
model.add_reactions([phenol_transport])
model.add_reactions([phenol_monooxygenase])

Add guaiacol reactions for exchange, transport, and [conversion to catechol](https://www.genome.jp/entry/R12301) 

In [6]:
guaiacol_exchange = cobra.Reaction('EX_guaiacol_e')
guaiacol_exchange.name = 'Guaiacol exchange'
guaiacol_exchange.lower_bound = -1000.  
guaiacol_exchange.upper_bound = 1000.  
guaiacol_exchange.add_metabolites({
    guaiacol_e: -1.0,
})

guaiacol_transport = cobra.Reaction('guaiacol_transport')
guaiacol_transport.name = 'Guaiacol transport'
guaiacol_transport.lower_bound = -1000. 
guaiacol_transport.upper_bound = 1000. 
guaiacol_transport.add_metabolites({
    guaiacol_e: -1.0,
    guaiacol_c:  1.0,
}) 

# EC 1.14.14.-
guaiacol_o_demethylase = cobra.Reaction('GUADEM')
guaiacol_o_demethylase.name = 'Guaicaol O-demethylase'
guaiacol_o_demethylase.lower_bound = 0.
guaiacol_o_demethylase.upper_bound = 1000.  # This is the default
guaiacol_o_demethylase.add_metabolites({
    guaiacol_c: -1.0,
    model.metabolites.get_by_id('nadph_c'):   -1.0,
    model.metabolites.get_by_id('o2_c'): -1.0,
    model.metabolites.get_by_id('catechol_c'): 1.0,
    model.metabolites.get_by_id('fald_c'):  1.0,
    model.metabolites.get_by_id('nadp_c'):  1.0,
    model.metabolites.get_by_id('h2o_c'):  1.0,
    
}) 

model.add_reactions([guaiacol_exchange])
model.add_reactions([guaiacol_transport])
model.add_reactions([guaiacol_o_demethylase])

Ensure that the TCA cycle is complete

In [7]:
model.reactions.get_by_id('SUCD1').upper_bound = 1000
model.reactions.get_by_id('SUCD1')

Reaction identifier,SUCD1
Name,Succinate dehydrogenase
Memory address,0x07f0fca52ee10
Stoichiometry,fad_c + succ_c <=> fadh2_c + fum_c Flavin adenine dinucleotide oxidized + Succinate <=> Flavin adenine dinucleotide reduced + Fumarate
GPR,WP_005240907_1 or ( WP_005247561_1 and WP_005247564_1 and WP_005247566_1 ) or ( WP_005239878_1 an...
Lower bound,-1000.0
Upper bound,1000


#### Define minimal media where the only carbon containing inputs are carbon sources

In [8]:
model.medium = {
 'EX_glc__D_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
#  'EX_tag': 0.0
}

Set some bounds to prevent known futile cycles

In [9]:
for r_id in ['HADPCOADH3']:
    model.reactions.get_by_id(r_id).lower_bound = 0
    model.reactions.get_by_id(r_id).upper_bound = 0

#### Check that model can grow with each carbon source

In [10]:
# glucose
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 10.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of glucose can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
10.0 EX_glc__D_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10 mmol of glucose can make 0.440 g of biomass


In [11]:
# phenol
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 10.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of phenol can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10.0 EX_phenol_e
10 mmol of phenol can make 0.227 g of biomass


In [12]:
# vanillate
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 10.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of vanillate can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
10.0 EX_vanlt_e
1000.0 EX_zn2_e
10 mmol of vanillate can make 0.198 g of biomass


In [13]:
# 4-hydroxybenzoate
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 10.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of 4-hydroxybenzoate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_4hbz_e
1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10 mmol of 4-hydroxybenzoate can make 0.198 g of biomass


In [14]:
# guaiacol
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 10.0
    medium["EX_bz_e"] = 0.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of guaiacol can make {solution.objective_value:.3f} g of biomass')

1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10.0 EX_guaiacol_e
10 mmol of guaiacol can make 0.227 g of biomass


In [15]:
# benzoate
with model:
    medium = model.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_phenol_e"] = 0.0
    medium["EX_vanlt_e"] = 0.0
    medium["EX_4hbz_e"] = 0.0
    medium["EX_guaiacol_e"] = 0.0
    medium["EX_bz_e"] = 10.0
    model.medium = medium
    [print(model.medium[m], m) for m in model.medium]
    solution = model.optimize()
    print(f'10 mmol of benzoate can make {solution.objective_value:.3f} g of biomass')

10.0 EX_bz_e
1000.0 EX_ca2_e
1000.0 EX_cl_e
1000.0 EX_cobalt2_e
1000.0 EX_cu2_e
1000.0 EX_fe2_e
1000.0 EX_fe3_e
1000.0 EX_h2o_e
1000.0 EX_h_e
1000.0 EX_k_e
1000.0 EX_mg2_e
1000.0 EX_mn2_e
1000.0 EX_mobd_e
1000.0 EX_nh4_e
1000.0 EX_o2_e
1000.0 EX_pi_e
1000.0 EX_so4_e
1000.0 EX_zn2_e
10 mmol of benzoate can make 0.227 g of biomass


#### Save model

In [17]:
model.id = 'ropacus_model_A'
model.name = 'R. opacus model A'
cobra.io.write_sbml_model(model, "../models/ropacus_model_A.xml")
model

Name,ropacus_model_A
Memory address,0x07f0f21419c90
Number of metabolites,1951
Number of reactions,3017
Number of groups,0
Objective expression,1.0*Growth - 1.0*Growth_reverse_699ae
Compartments,"cytosol, periplasm, extracellular space"
